### Q1: Turbit primarily deals with time-series data generated by sensors for temperature, pressure and other quantities. Which modern time-series forecasting approach would you choose to model such data? Describe its advantages and disadvantages. How would you implement it using tensorflow?

I would come up with several possible approaches in my mind:

LSTM: It's a very flexible model to take different features across different time steps to make univariate or multivariate predictions. Also it enables the model to learn temporal information and train easily by providing a "gradient highway". More importantly, most DL libraries (Tensorflow, Keras, Pytorch) provide an implementation of LSTM, making it easier to use. But it can also be difficult to tune as there're many parameters regarding the model structure. And also it can be difficult for LSTM to capture long-term dependencies if we're dealing with time-series data across a long period.

N-BEATS & N-BEATSX: N-BEATS is a very powerful forecasting model I've played around with in my study. It requires basically no domain knowledge and is also fast to train. For most time-series benchmark datasets it has achieved excellent results. The problem regarding N-BEATS is that it only focuses on univariate time series forecasting, which is to say it only takes one feature and outputs predicitions on the same feature.
N-BEATSX, however, takes the external factors also as input and tries to incorporate more information into the model. It also shows promising results and has already been applied into realistic industry scenarios. General drawbacks of these two models are there're no embedded implementation in DL libraries and requires much more effort to implement. Also training these two models also require more computing resources.

Links for N-BEATS and N-BEATSX paper:

https://arxiv.org/pdf/1905.10437.pdf?trk=public_post_comment-text

https://www.sciencedirect.com/science/article/pii/S0169207022000413

Links for N-BEATS and N-BEATSX implementation:

https://github.com/philipperemy/n-beats

https://github.com/cchallu/nbeatsx

General words on implementation using TensorFlow:
In my opinion it's always good practice to divide the entire codebase into one file for preprocessing data into the required format for model as different customers may have different formats, one file for building the model and one file for training and evaluating the model where we can pass different hyperparameters including the model to use. Regarding the model implementation I think it would make sense always to start with simple part and build further on them (such as the block in N-BEATS).

### Q2: We often deal with data gaps. What approaches can you use to deal with missing data in the context of time-series? What are the advantages and disadvantages?

To deal with missing data, a simple approach is to just replace the missing values with constants such as zeros, median of the feature, mean of the feature. These approaches are extremely simple, easy and cheap to implement. But they don't preserve the changing trend of the time series data and pollutes the time series as a null value DOESN'T mean zero/ median/ mean.

A more appropriate approach would be to use interpolation. The following are some interpolation methods and their pros and cons in my opinion.

Linear Interpolation:
It uses the nearest past point and nearest next point to simulate a linear function to interpolate the missing value. It's also very easy and simple to implement and also preserves the changing trend of time series data. It can work well for a dense time series but for a wide gap it might not fit well as it always assumes a constant change rate of data.

Nearest Neighbor Interpolation:
Similar to linear interpolation, it also looks into the nearest past and next point. But it then just takes either the nearest past value (nearest past interpolation) or  the nearest next value (next forward interpolation). This is also a very easy method but it can lead to step-like discontinuous time series data.

Weighted Nearest Neighbor Interpolation:
This method takes several nearest neighbors into account and calculated a weighted interpolation based on the distance to the missing value. This can overcome the drawbacks of a simple NN Interpolation but it can be a bit more expensive to calculate and also there's a hyperparameter needed to be chosen (number of neighbors).

Polynomial Interpolation:
As a more general variant of linear interpolation where p is 1, polynomial interpolation uses (p-1) neighboring points to fit a p-degree polynomial function. This could resolve the issues of linear interpolation while the computing cost could be relatively high.

In my opinion both smoothness and sensitivity of the interpolation are important factors. I also came across these similar concepts during my master study from the paper below:
https://arxiv.org/pdf/2102.11409.pdf


### Q3: When reporting turbine anomalies, customers usually want to know the underlying reason for the unusual data points. Imagine that a thermometer in the gearbox of a turbine suddenly started reporting temperatures ten degrees higher than expected. List possible causes for such a pattern and how Turbit could distinguish between them?

Following are some possible reasons and what we could do:

The first reason could be that there's a sudden change of weather condition in the area of where the turbine is located in. We could check the data from other internal and external temperature sensors to see if there's also a sudden change in them. In case there's no other temperature sensor available we can also look for the weather information online about the area to double check.

The second reason could be that the thermometer itself is broken and sending wrong temperatures. In this situation we need to first check other internal sensors for similar changing pattern. If all other sensors seem to perform normally as before, we could subtract ten degrees from the current temperature and pass the expected temperature to our learned model (let's say we again predict the power) for couple of rounds of inference. If the model output is close to the actual ground truth we're observing, then we can be pretty sure that only the thermometer is broken.

Another reason could be that the thermometer is not broken and there's indeed something abnormal going on to increase the temperature. In this case we should also start from checking other sensor data/ features we have in hand to compare the correlation between the temperature with other factors before and after the sudden change in the temperature sensor. If we manage to find a change in correlation between temperature sensor and other sensors, we could narrow down the problem to certain reasons (maybe abnormal workload or a broken hardware in the turbine etc.).